<a href="https://colab.research.google.com/github/SanjayBista1010/DeepLearning/blob/main/PytorchSnake%26Spider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

# Path to your ZIP file in Google Drive
zip_path = '/content/drive/MyDrive/dataset.zip'

# Destination folder in Colab
extract_path = '/content/images'
os.makedirs(extract_path, exist_ok=True)

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extracted files to {extract_path}")


Extracted files to /content/images


In [17]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import Subset, DataLoader
from tqdm import tqdm

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes=2):  # 2 classes: snake, spider
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 32 * 32, num_classes)  # flatten size updated

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
input_size = 16_384  # after 128x128 images and 2x2 pooling
num_classes = 2      # snake/spider
learning_rate = 0.001
batch_size = 32      # or 64 if memory allows
num_epochs = 10      # increase if needed

In [20]:
!pip install opencv-python

In [21]:
import cv2
from PIL import Image
import numpy as np


class ToBW:
    def __call__(self, img):
        # Convert PIL image to numpy array
        img_np = np.array(img)
        # Convert to grayscale using OpenCV
        bw = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
        # Convert back to PIL Image
        img_pil = Image.fromarray(bw)
        return img_pil

# Use in a transform pipeline
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    ToBW(),                # custom OpenCV B&W conversion
    transforms.ToTensor()
])


In [22]:
dataset_path = 'images/dataset'  # contains 'snake' and 'spider' subfolders
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
print(f"Classes: {full_dataset.classes}")  # ['snake', 'spider']

Classes: ['snake', 'spider']


In [23]:
from sklearn.model_selection import train_test_split


indices = list(range(len(full_dataset)))
labels = [full_dataset[i][1] for i in indices]

train_indices, test_indices = [], []

for class_label in [0, 1]:  # 0=snake, 1=spider
    class_indices = [i for i, l in enumerate(labels) if l == class_label]

    # train_test_split to select fixed numbers
    train_idx, test_idx = train_test_split(
        class_indices, train_size=2400, test_size=200, random_state=42, shuffle=True
    )
    train_indices += train_idx
    test_indices += test_idx

In [24]:
train_dataset = Subset(full_dataset, train_indices)
test_dataset = Subset(full_dataset, test_indices)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")


Train samples: 4800, Test samples: 400


In [25]:
model = CNN(in_channels =1, num_classes=num_classes).to(device)

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [27]:
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        #move data and targets to the device (GPU/CPU)
        data = data.to(device)
        targets = targets.to(device)

        #forward pass: compute the model output
        scores = model(data)
        loss = criterion(scores, targets)

        #backward pass: compute the gradients
        optimizer.zero_grad()
        loss.backward()

        #optimization step: update the model parameters
        optimizer.step()

Epoch [1/10]


100%|██████████| 150/150 [00:29<00:00,  5.09it/s]


Epoch [2/10]


100%|██████████| 150/150 [00:25<00:00,  5.92it/s]


Epoch [3/10]


...

100%|██████████| 150/150 [00:25<00:00,  5.95it/s]


Epoch [9/10]


100%|██████████| 150/150 [00:25<00:00,  5.93it/s]


Epoch [10/10]


100%|██████████| 150/150 [00:24<00:00,  6.17it/s]


In [29]:
def check_accuracy(loader, model, loader_name="Data"):
    print(f"Checking accuracy on {loader_name}")

    num_correct = 0
    num_samples = 0
    model.eval()  # evaluation mode

    with torch.no_grad():  # no gradient needed
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()  # convert to number
            num_samples += predictions.size(0)

    accuracy = 100 * num_correct / num_samples
    print(f"Got {num_correct}/{num_samples} correct -> Accuracy: {accuracy:.2f}%")

    model.train()  # back to training mode

# Usage
check_accuracy(train_loader, model, loader_name="Training Data")
check_accuracy(test_loader, model, loader_name="Test Data")

Checking accuracy on Training Data
Got 3736/4800 correct -> Accuracy: 77.83%
Checking accuracy on Test Data
Got 291/400 correct -> Accuracy: 72.75%
